In [54]:
!wget https://2mp.conae.gov.ar/descargas/Coberturas/IGN%20-%20SIG250/001b_departamentos.zip

--2015-10-25 17:50:08--  https://2mp.conae.gov.ar/descargas/Coberturas/IGN%20-%20SIG250/001b_departamentos.zip
Resolviendo 2mp.conae.gov.ar (2mp.conae.gov.ar)... 200.16.80.212
Conectando con 2mp.conae.gov.ar (2mp.conae.gov.ar)[200.16.80.212]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 7977210 (7,6M) [application/zip]
Grabando a: “001b_departamentos.zip”

100%[======================================>] 7.977.210   1,33MB/s   en 6,9s   

2015-10-25 17:50:15 (1,11 MB/s) - “001b_departamentos.zip” guardado [7977210/7977210]



In [56]:
!unzip 001b_departamentos.zip

Archive:  001b_departamentos.zip
  inflating: departamentos.dbf       
  inflating: departamentos.prj       
  inflating: departamentos.shp       
  inflating: departamentos.shx       


In [ ]:
!pip install pyshp

In [1]:
import shapefile

#I got a few cribs for how to use this package to generate JOSN file from here
#https://github.com/mlaloux/PyShp-as-Fiona--with-geo_interface-
def records(filename):  
    # generator 
    reader = shapefile.Reader(filename)  
    fields = reader.fields[1:]  
    field_names = [field[0] for field in fields]  
    for sr in reader.shapeRecords():  
        geom = sr.shape.__geo_interface__
        atr = dict(zip(field_names, sr.record))  
        try:
            geom = {k.decode('utf8'):v.strip().decode('utf8') for k,v in geom.items() if not isinstance(v, tuple) and k.strip()}               
            atr = {k.decode('utf8'):v.strip().decode('utf8') for k,v in atr.items() if not isinstance(v, tuple) and  k.strip()}               
        except AttributeError:
            pass
        yield dict(geometry=geom,properties=atr)
        
import json

def geojsonify(shpfile):
    features=[row for row in records(shpfile)]
    for row in features:
        row["type"]="Feature"
    return dict(type='FeatureCollection',features=features)

In [23]:
geo = geojsonify('departamentos.shp')

In [24]:
# simplifico propiedades
properties = {'provincia':'provincia', 'departa': 'departamento', 'cabecer': 'cabecera', 'objectid': 'id'}

In [25]:
for f in geo['features']:
    props = {}
    for k,v in f['properties'].items():
        k = properties.get(k, None)
        if not k:
            continue
        props[k] = v
        if isinstance(v, str) and 'CIUDAD AU' in v:
            props[k]  = 'CIUDAD AUTONOMA DE BUENOS AIRES'
        if isinstance(v, bytes):
            props[k] = '' if not v.strip() else v.decode('latin1')
    f['properties'] = props

In [19]:
geo['features'][0]

{'geometry': {'coordinates': (((-58.32487869300002, -30.838373183999977),
    (-58.320339203, -30.838726043999998),
    (-58.31584930399998, -30.838579177999975),
    (-58.311347961000024, -30.83832931500001),
    (-58.306930541999975, -30.83895874000001),
    (-58.30253601099997, -30.839849472000026),
    (-58.298187256000006, -30.84085655199999),
    (-58.29391479499998, -30.842073441000025),
    (-58.28979873700001, -30.843627930000025),
    (-58.28535461400003, -30.844341278),
    (-58.28133010900001, -30.846097945999986),
    (-58.27799606299999, -30.84868621800001),
    (-58.27512741100003, -30.85177802999999),
    (-58.273265838999976, -30.855222701999992),
    (-58.27278900099998, -30.85909652700002),
    (-58.27243041999998, -30.863012314000002),
    (-58.27181243899997, -30.866865157999996),
    (-58.271484375, -30.870729445999984),
    (-58.27149963400001, -30.874614715999996),
    (-58.27020645099998, -30.878313064999986),
    (-58.26895141599999, -30.882076262999988),
    

In [26]:
provs = set([f['properties']['provincia'] for f in geo['features']])
provs

{'BUENOS AIRES',
 'CATAMARCA',
 'CHACO',
 'CHUBUT',
 'CIUDAD AUTONOMA DE BUENOS AIRES',
 'CORDOBA',
 'CORRIENTES',
 'ENTRE RIOS',
 'FORMOSA',
 'JUJUY',
 'LA PAMPA',
 'LA RIOJA',
 'MENDOZA',
 'MISIONES',
 'NEUQUEN',
 'RIO NEGRO',
 'SALTA',
 'SAN JUAN',
 'SAN LUIS',
 'SANTA CRUZ',
 'SANTA FE',
 'SANTIAGO DEL ESTERO',
 'TIERRA DEL FUEGO',
 'TUCUMAN'}

In [27]:
json.dump(geo, open('departamentos-argentina.json', 'w'),  indent=1)

In [28]:
!head departamentos-argentina.json

{
 "features": [
  {
   "type": "Feature",
   "properties": {
    "id": 1,
    "departamento": "CONCORDIA",
    "cabecera": "CONCORDIA",
    "provincia": "ENTRE RIOS"
   },


In [29]:
for p in provs:
    prov_geo = geo.copy()
    features = [f for f in prov_geo['features'] if f['properties']['provincia'] == p]
    prov_geo['features'] = features
    json.dump(prov_geo, open('departamentos-%s.json' % p.lower().replace(' ', '_'), 'w'),  indent=1)

In [30]:
geojsons = !ls departamentos*.json

In [31]:
for f in geojsons:
    !topojson -o {f.replace('json','topojson')} {f}